# Лабораторная работа № 5
## Хапчаев Тимур Русланович M32071
### Вариант 4

In [70]:
import pandas as pd
import numpy as np
from scipy.stats import t, chi2, f

In [71]:
data = pd.read_csv("/Users/tkhapchaev/Documents/GitHub/mathematical-statistics/lab-5/song_data.csv")

Пусть $X$ - матрица данных с $n$ строками и $k$ столбцами, где $k$ - общее количество регрессоров, $n$ - объем выборок, в которой i-я строка представляет собой ряд i-х наблюдений для каждого из регрессоров (здесь $k = 4$). Столбец данной матрицы с номером j представлет собой набор $n$ наблюдений для j-го по порядку регрессора.
<br>
<br>
Линейная модель имеет вид:<br>
<br>
$y = X\beta + \epsilon$
<br>
Здесь $y$ - зависимая переменная, $\beta$ - вектор истинных коэффициентов модели, $\epsilon$ - случайная ошибка модели.
<br>
Вектор МНК-оценок коэффициентов имеет вид: $b = (X^TX)^{-1} X^Ty$
Ковариационная матрица оценок коэффициентов: $\text{Var}(b) = \sigma^2 (X^TX)^{-1}$
Её состоятельная оценка: $\hat{\text{Var}(b)} = s^2(X^TX)^{-1}$, где $s^2 = \dfrac{\text{SSR}}{n-k}$, $\text{SSR}$ - сумма квадратов остатков регрессии, $\sigma^2$ - теоретическая дисперсия случайной ошибки модели.

In [72]:
n = data.shape[0]
C = np.ones(n).reshape((n, 1))

duration = np.array(data["song_duration_ms"]).reshape((n, 1))
danceability = np.array(data["danceability"]).reshape((n, 1))
energy = np.array(data["energy"]).reshape((n, 1))
y =  np.array(data["song_popularity"]).reshape((n, 1))

X = np.hstack((C, duration, danceability, energy))

Вектор МНК-оценок коэффициентов:

In [73]:
b = np.linalg.inv((X.T @ X)) @ X.T @ y

Сумма квадратов остатков регрессии:

In [74]:
y_pred = X @ b
SSR = np.sum((y - y_pred)**2)

Ковариационная матрица вектора МНК-оценок:

In [75]:
var_b = (SSR / (n - 4)) * np.linalg.inv((X.T @ X))

Оцениваемая модель: <br>
<br>
$\text{popularity} = 44.61 - 0.000003\cdot \text{duration} + 14.478\cdot \text{danceability} - 0.257 \cdot \text{energy}$<br>
<br>
Оценка остаточной дисперсии: $s^2 = \dfrac{\text{SSR}}{n-k} \approx 474.681$

Построим доверительные интервалы:
<br>
Доверительный интервал для коэффициентов модели с вероятностью накрытия истинного значения параметра в $1 - \alpha$:<br>
<br>
$(b_i - q\cdot\sqrt{\text{Var}(b)_i}; \ b_i + q\cdot\sqrt{\text{Var}(b)_i})$, где $q$ - квантиль распределения Стьюдента с $n - k$ степенями свободы уровня $1 - \alpha/2$,
 $b_i$ - оценка i-го коэффициента модели,
 $\text{Var}(b)_i$ - соответствующая оценка дисперсии оценки i-го коэффициента модели.<br>
<br>
Доверительный интервал для дисперсии случайной ошибки с вероятностью накрытия истинного значения параметра в $1 - \alpha$:<br>
<br>
$(\dfrac{\text{SSR}}{q_{\chi, 1 - \alpha/2}}; \ \dfrac{\text{SSR}}{q_{\chi, \alpha/2}})$, где $q_{\chi, \beta}$ - квантиль хи-квадрат распределения с $n-k$ степенями свободы уровня $\beta$.

Стандартные ошибки оценок коэффициентов:

In [76]:
se_const, se_x1, se_x2, se_x3 = np.sqrt(var_b.diagonal()[0]), np.sqrt(var_b.diagonal()[1]), np.sqrt(var_b.diagonal()[2]), np.sqrt(var_b.diagonal()[3])

b = b[0][0], b[1][0], b[2][0], b[3][0]

Доверительные интервалы для коэффициентов модели:

In [77]:
alpha = 0.05 # 95 %

const_1 = b[0] - t.ppf(1 - alpha / 2, df = n - 4) * se_const
const_2 = b[0] + t.ppf(1 - alpha / 2, df = n - 4) * se_const

duration_1 = b[1] - t.ppf(1 - alpha / 2, df = n - 4) * se_x1
duration_2 = b[1] + t.ppf(1 - alpha / 2, df = n - 4) * se_x1

danceability_1 = b[2] - t.ppf(1 - alpha / 2, df = n - 4) * se_x2
danceability_2 = b[2] + t.ppf(1 - alpha / 2, df = n - 4) * se_x2

energy_1 = b[3] - t.ppf(1 - alpha / 2, df = n - 4) * se_x3
energy_2 = b[3] + t.ppf(1 - alpha / 2, df = n - 4) * se_x3

print(f'Доверительный интервал для константы: ({const_1}; {const_2})')
print(f'Доверительный интервал для коэффициента перед song_duration_ms: ({duration_1}; {duration_2})')
print(f'Доверительный интервал для коэффициента перед danceability: ({danceability_1}; {danceability_2})')
print(f'Доверительный интервал для коэффициента перед energy: ({energy_1}; {energy_2})')

Доверительный интервал для дисперсии случайной ошибки:

In [78]:
pe_1 = SSR / chi2.ppf(1 - alpha / 2, df = n - 4)
pe_2 = SSR / chi2.ppf(alpha / 2, df = n - 4)

print(f'Доверительный интервал для дисперсии случайной ошибки: ({pe_1}; {pe_2})')

Имеем 95% доверительные интервалы:<br>
<br>
Для константы: (42.64; 46.579)<br>
Для коэффициента перед song_duration_ms: (-0.0000081; 0.00000239)<br>
Для коэффициента перед danceability: (12.479; 16.478)<br>
Для коэффициента перед energy: (-1.719;  1.205)<br>
Для дисперсии случайной ошибки: (465.236; 484.416)<br>

Коэффициент детерминации:

$R^2 = 1 - \dfrac{\text{SSR}}{\text{TSS}}$, где $\text{TSS} = \sum_{i=1}^{n} (y_i - \overline{y})^2$, и $\overline{y} = \dfrac{1}{n} \sum_{i=1}^{n} y_i$

In [79]:
TSS = np.sum((y-np.mean(y))**2)
R2 = 1 - SSR/TSS

In [80]:
R2_rounded = round(R2, 3)
print(R2_rounded)

Гипотезы из условия:<br>
Чем больше энергичность, тем больше популярность
$H_0: \beta_3 \leq 0$, $H_1: \beta_3 > 0$, $\beta_3$ (коэффициент при energy в модели)<br>
<br>
Популярность зависит от продолжительности
$H_0: \beta_1 = 0$, $H_1: \beta_1 \neq 0$, $\beta_1$ (коэффициент при song_duration_ms в модели)<br>
<br>
Популярность зависит от энергичности и танцевальности
$H_0: \beta_2 = \beta_3 = 0$, $H_1: \text{хотя бы один из коэффициентов}\ \beta_2, \beta_3\ \text{отличен от}\ 0$, $\beta_2, \beta_3$ - коэффициенты при danceability и energy соответственно

Первые две гипотезы на уровне значимости $\alpha$ проверим с помощью t-статистики:<br>
$t = \dfrac{b_i}{\sqrt{\text{Var}(b)_i}}$, где в первом случае нулевая гипотеза отвергается, если значение статистики превышает квантиль уровня $1 - \alpha$ распределения Стьюдента с $n-k$ степенями свободы, во втором случае - если значение статистики по модулю превышает квантиль уровня $1 - \alpha/2$ распределения Стьюдента с $n-k$ степенями свободы.<br>
<br>
Третью гипотезу проверим с помощью F-статистики:<br>
$F = \dfrac{(R^2_u - R^2_r)/m}{(1 - R^2_u)/(n-k)}$, где $R^2_u, R^2_r$ - коэффициенты детерминации соответственно в модели без ограничений и модели с ограничениями, $m$ - количество ограничений (здесь $m = 2$).<br>
Нулевая гипотеза отвергается на уровне значимости $\alpha$, если значение статистики превышает квантиль уровня $1-\alpha$ распределения Фишера со степенями свободы $m$ и $n-k$

In [81]:
t_energy = b[3] / se_x3
t_duration = b[1] / se_x1

In [82]:
t_energy > t.ppf(0.95, df = n - 4)

Таким образом, гипотеза о том, что $\beta_3 \leq 0$ не отвергается на уровне значимости 0.05, то есть нельзя утверждать, что чем больше энергичность песни, тем больше её популярность (с максимальной вероятностью ошибки первого рода в 0.05)

In [83]:
abs(t_duration) > t.ppf(0.975, df = n-4)

Таким образом, гипотеза о том, что $\beta_1 = 0$ не отвергается на уровне значимости 0.05, то есть можно утверждать, что популярность песни не зависит от продолжительности (с максимальной вероятностью ошибки первого рода в 0.05)

Посчитаем $R^2_r$, для этого посчитаем коэффициент детерминации в модели с ограничениями (в модели, где в качестве независимых переменных выступают константа и продолжительность песни):

In [84]:
X_restricted = np.hstack((C, duration))

b_restricted = np.linalg.inv(X_restricted.T @ X_restricted) @ X_restricted.T @ y
y_restricted = X_restricted @ b_restricted

SSR_restricted = np.sum((y - y_restricted)**2)
R2_restricted = 1 - SSR_restricted/TSS

In [85]:
F = ((R2 - R2_restricted) / 2) / ((1 - R2) / (n - 4))

In [86]:
F > f.ppf(0.95, dfn = 2, dfd = n - 4)

Таким образом, гипотеза о том, что популярность песни не зависит от энергичности и танцевальности отвергается на уровне значимости 0.05, то есть можно утверждать, что популярность песни зависит одновременно от энергичности и танцевальности (с максимальной вероятностью ошибки первого рода в 0.05)